# Time Series classification

### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/AN2DL/Homework2

## Setup the environment

### Import libraries

In [ ]:
import numpy as np
import os
import time
from pathlib import Path
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler

import joblib

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Dropout,BatchNormalization,Activation,Dense,Concatenate,Add,Average
from keras.layers import GlobalAveragePooling1D,Bidirectional,LSTM,Conv1D,MaxPooling1D
from keras.models import Model

print(f'TensorFlow version: {tf.__version__}')

### Setup Keras Tuner

In [ ]:
!pip install -q -U keras-tuner
import keras_tuner as kt

#LOG_DIR = Path() / 'kerastuner_logs' / str(int(time.time()))
LOG_DIR = Path() / 'kerastuner_logs' / '1671115788'
LOG_DIR.mkdir(parents=True, exist_ok=True)

### Load Data

In [ ]:
#!unzip 'training_dataset_homework2'
X_data = np.load('x_train.npy')
y_data = keras.utils.to_categorical(np.load('y_train.npy'))

In [ ]:
print(f'{X_data.shape=}')
print(f'{y_data.shape=}')
print(f'We have {X_data.shape[0]} samples, each of which is a time series with {X_data.shape[1]} time steps and {X_data.shape[2]} features. We have to classify {y_data.shape[1]} classes.')

### Configuration variables

In [ ]:
BATCH_SIZE = 16
SEED = 42
tf.random.set_seed(SEED)
MODELS_DIR = Path() / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

REPORT_DIR = Path() / 'report_material'
REPORT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_SHAPE = X_data.shape[1:]
NUM_CLASSES = y_data.shape[-1]

EPOCHS = 200

### Data exploration

In [ ]:
label_mapping = {
    "Wish" : 0,
    "Another" : 1,
    "Comfortably" : 2,
    "Money" : 3,
    "Breathe" : 4,
    "Time" : 5,
    "Brain" : 6,
    "Echoes" : 7,
    "Wearing" : 8,
    "Sorrow" : 9,
    "Hey" : 10,
    "Shine" : 11
}

LABELS = list(label_mapping.keys())

In [ ]:
y_ints = [y.argmax() for y in y_data]
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_ints),
                                                  y=y_ints)

class_weights = dict(zip(np.unique(y_ints), class_weights))
#class_weights = dict(zip(label_mapping.keys(), class_weights))
class_weights

In [ ]:
unique, counts = np.unique(y_ints, return_counts=True)
counts

plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')

fig, ax = plt.subplots(figsize=(8,1.5))

plt.bar(range(len(counts)), list(counts), align='center')
plt.xticks(range(len(label_mapping)), list(label_mapping.keys()))

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
plt.setp(ax.get_yticklabels(), rotation=0, ha='right', rotation_mode='anchor')

filename = f'class_imbalance.pdf'
plt.savefig(str(REPORT_DIR / filename), bbox_inches='tight')

plt.show()

#### Plot a datum

In [ ]:
NUM_FEATURES = X_data.shape[2]
DATUM = 5

In [ ]:
plt.style.use('seaborn-whitegrid')
#plt.style.use('ggplot')

for i in range(NUM_FEATURES):
    ax = plt.subplot(3, 2, i + 1)
    ax.plot(X_data[DATUM,:,i])
    #ax.grid()
    ax.set_title(f'Feature {i+1}')
    #ax.set_xlabel("t")

plt.tight_layout()
filename = f'datum_{DATUM}.pdf'
plt.savefig(str(REPORT_DIR / filename), bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('seaborn-whitegrid')

fig, ax = plt.subplots()
    
ax.plot(X_data[5])

fig.tight_layout()
#plt.savefig(str(REPORT_DIR / filename), bbox_inches='tight')
plt.show()

### Train / Validation split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.15, random_state=SEED)

### Normalization

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

In [ ]:
print(scaler.mean_)
print(np.sqrt(scaler.var_))

In [ ]:
# save scaler
joblib.dump(scaler, Path() / 'submission' / 'scaler.gz')

### Data Augmentation

If the following parameter is set to `True` all the models will be run with data augmentation, except the Keras tuner one.

In [ ]:
AUGMENTATION = True

#### Augmentation library

In [ ]:
!pip install tqdm

From this [GitHub repo](https://github.com/uchidalab/time_series_augmentation).

In [ ]:
import utils.datasets as ds
import utils.augmentation as aug
import utils.helper as hlp

#### Augmentation visualizations

In [ ]:
FEATURE = 3

In [ ]:
plt.rc('font', size=10)
plt.style.use('seaborn-whitegrid')

sample_1d = X_train[DATUM,:,FEATURE] # sample 5, feature 3, from the normalized dataset
sample_3d = np.expand_dims(sample_1d, axis=0)
sample_3d = np.expand_dims(sample_3d, axis=2)

fig, ax = plt.subplots(3, 2)

ax[0,0].plot(sample_1d)
ax[0,0].set_title(f'Original')

temp = aug.jitter(sample_3d)[0,:,0]
ax[0,1].plot(sample_1d)
ax[0,1].plot(temp, linewidth = 1)
ax[0,1].set_title(f'Jitter')

temp = aug.scaling(sample_3d)[0,:,0]
ax[1,0].plot(sample_1d)
ax[1,0].plot(temp, linewidth = 1)
ax[1,0].set_title(f'Scaling')

temp = aug.permutation(sample_3d)[0,:,0]
ax[1,1].plot(sample_1d)
ax[1,1].plot(temp, linewidth = 1)
ax[1,1].set_title(f'Permutation')

temp = aug.window_slice(sample_3d)[0,:,0]
ax[2,0].plot(sample_1d)
ax[2,0].plot(temp, linewidth = 1)
ax[2,0].set_title(f'Window Slicing')

temp = aug.time_warp(sample_3d)[0,:,0]
ax[2,1].plot(sample_1d)
ax[2,1].plot(temp, linewidth = 1)
ax[2,1].set_title(f'Time Warp')

plt.tight_layout()
filename = 'augmentation_vis.pdf'
plt.savefig(str(REPORT_DIR / filename), bbox_inches='tight')
plt.show()

#### Custom Augmentation Layer

In [ ]:
class data_augmentation(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, inputs, training=None):
        if training:
            inputs = inputs.numpy()
            inputs = aug.scaling(inputs)
            inputs = aug.permutation(inputs)
        return inputs

### Utility functions

In [ ]:
def evaluate_model(model, eval_data, eval_y_true, normalized=True, sig_fig=4, save=False, filename='confusion'):
    
    # get the predictions
    eval_y_pred = np.argmax(model.predict(eval_data, verbose=0), axis=-1)
    eval_y_true = np.argmax(eval_y_true, axis=-1)
    
    # compute the confusion matrix
    cm = confusion_matrix(eval_y_true, eval_y_pred)
    
    # normalize if needed
    if normalized:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # compute classification metrics
    accuracy = accuracy_score(eval_y_true, eval_y_pred)
    f1 = f1_score(eval_y_true, eval_y_pred, average=None)
    print('Accuracy:', np.round(accuracy, sig_fig))
    print('F1-scores:')
    for i,label in enumerate(LABELS):
        print(f'{label:<14} {np.round(f1[i], sig_fig)}')
    
    plt.rc('font', size=6)

    ax = plt.subplot()

    sns.heatmap(cm, cmap='Blues', annot=True)  # annot=True to annotate cells, fmt='g' to disable scientific notation

    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    #ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(LABELS)
    ax.yaxis.set_ticklabels(LABELS)
    

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
    plt.setp(ax.get_yticklabels(), rotation=0, ha='right', rotation_mode='anchor')
    
    if save:
        filename = filename + '.pdf'
        plt.savefig(str(REPORT_DIR / filename), bbox_inches='tight')
    
    plt.show()

def plot_history(history, save=False, filename='history', show_lr=True):
    """Prints the diagnostic plot"""
    
    plt.style.use('default')
    #plt.style.use('ggplot')
    plt.rc('font', size=8)
    
    if show_lr:
        fig, (ax1,ax2,ax3) = plt.subplots(3, sharex=True)
    else:
        fig, (ax1,ax2) = plt.subplots(2, sharex=True)
    
    #fig.suptitle('Training and validation')
    
    best_epoch = np.argmin(history.history['val_loss'])

    ax1.plot(history.history['accuracy'], label='Training accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation accuracy')
    ax1.axvline(x=best_epoch, label='Best Epoch (lowest loss)', alpha=.3, ls='--', color='#5a9aa5')
    ax1.set_ylabel('Accuracy')
    #ax1.set_xlabel('Epochs')
    ax1.legend(loc='upper left')
    ax1.grid(alpha=.3)
    
    ax2.plot(history.history['loss'], label='Training loss')
    ax2.plot(history.history['val_loss'], label='Validation loss')
    ax2.axvline(x=best_epoch, label='Best Epoch (lowest loss)', alpha=.3, ls='--', color='#5a9aa5')
    ax2.set_ylabel('Loss')
    #ax2.set_xlabel('Epochs')
    ax2.legend(loc='upper left')
    ax2.grid(alpha=.3)
    
    if show_lr:
        ax3.plot(history.history['lr'], label='Learning Rate')
        ax3.axvline(x=best_epoch, label='Best Epoch (lowest loss)', alpha=.3, ls='--', color='#5a9aa5')
        ax3.set_ylabel('Learning Rate')
        ax3.set_xlabel('Epochs')
        ax3.legend(loc='upper left')
        ax3.grid(alpha=.3)
    
    if save:
        filename = filename + '.pdf'
        plt.savefig(str(REPORT_DIR / filename), bbox_inches='tight')
    
    plt.show()

In [ ]:
def train_model(model):
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[
            keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.5, min_lr=1e-5)
        ]
    )
    return history, model

In [ ]:
def reinitialize(model):
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))

## Training models

### Vanilla Long Short Term Memory (LSTM) Neural Network

In [ ]:
def build_LSTM_classifier(aug=False):

    input_layer = Input(shape=INPUT_SHAPE)
    x = input_layer
    
    if aug:
        x = data_augmentation()(x)

    # Feature extractor
    x = LSTM(128, return_sequences=True)(x)
    x = LSTM(128)(x)
    x = Dropout(.5)(x)

    # Classifier
    x = Dense(128, activation='relu')(x)
    
    output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

    # Connect input and output through the Model class
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy', run_eagerly=aug)

    # Return the model
    return model

In [ ]:
model = build_LSTM_classifier(aug=AUGMENTATION)

In [ ]:
history, model = train_model(model)

In [ ]:
plot_history(history, save=True, filename='history_lstm', show_lr=False)

In [ ]:
evaluate_model(model, X_val, y_val)

In [ ]:
model.save(MODELS_DIR / 'lstm-0.6877.h5')

In [ ]:
model = keras.models.load_model(MODELS_DIR / 'lstm-0.6877.h5', custom_objects={'data_augmentation': data_augmentation})

### Bidirectional Long Short Term Memory (BiLSTM) Neural Network

In [ ]:
def build_BiLSTM_classifier(aug=False):

    input_layer = Input(shape=INPUT_SHAPE)
    x = input_layer
    
    if aug:
        x = data_augmentation()(x)

    # Feature extractor
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(128))(x)
    x = Dropout(.5)(x)

    # Classifier
    x = Dense(128, activation='relu')(x)
    
    output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

    # Connect input and output through the Model class
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy', run_eagerly=aug)

    # Return the model
    return model

In [ ]:
model = build_BiLSTM_classifier(aug=AUGMENTATION)

In [ ]:
history, model = train_model(model)

In [ ]:
plot_history(history, save=True, filename='history_bilstm', show_lr=False)

In [ ]:
evaluate_model(model, X_val, y_val)

In [ ]:
model.save(MODELS_DIR / 'bilstm-0.7479.h5')

In [ ]:
model = keras.models.load_model(MODELS_DIR / 'bilstm-0.7479.h5', custom_objects={'data_augmentation': data_augmentation})

### 1D Convolutional Neural Network

In [ ]:
def build_1DCNN_classifier(aug=False):

    input_layer = Input(shape=INPUT_SHAPE)
    x = input_layer
    
    if aug:
        x = data_augmentation()(x)
    
    # Feature extractor
    filters = [2**i for i in range(6,10)]
    for f in filters:
        x = Conv1D(f,3,padding='same',activation='relu')(x)
        x = MaxPooling1D()(x)
    
    x = GlobalAveragePooling1D()(x)
    x = Dropout(.5)(x)

    # Classifier
    x = Dense(512, activation='relu')(x)
    x = Dropout(.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(.5)(x)
    
    output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

    # Connect input and output through the Model class
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy', run_eagerly=aug)

    # Return the model
    return model

In [ ]:
model = build_1DCNN_classifier(aug=AUGMENTATION)

In [ ]:
history, model = train_model(model)

In [ ]:
plot_history(history, save=True, filename='history_1dcnn', show_lr=False)

In [ ]:
evaluate_model(model, X_val, y_val)

In [ ]:
model.save(MODELS_DIR / '1dcnn-0.7452.h5')

In [ ]:
model = keras.models.load_model(MODELS_DIR / '1dcnn-0.7452.h5', custom_objects={'data_augmentation': data_augmentation})

### Keras tuner

In [ ]:
def model_builder(hp):

    # Build the neural network layer by layer
    input_layer = Input(shape=INPUT_SHAPE)
    x = input_layer

    # ---------------------------
    
    # best number of CNN1D layers
    num_cnn_layers = hp.Int('n_1dcnn_layers',0,8)
    filters = [2**i for i in range(6,6+num_cnn_layers)]
    for f in filters:
        x = Conv1D(f,3,padding='same',activation='relu')(x)
    
    # ---------------------------

    # best number of LSTM layers
    for i in range(hp.Int('n_lstm_layers',1,4)):
        x = Bidirectional(LSTM(
            hp.Int(f'LSTM_units_{i}', min_value=64, max_value=256, step=64), return_sequences=True))(x)

    x = GlobalAveragePooling1D()(x)

    x = Dropout(rate=hp.Choice(f'dropout_post_GAP_rate', values=[0.0,0.5]))(x)

    # ---------------------------

    # best number of Dense layers
    for i in range(hp.Int('n_dense_layers',1,2)):
        x = Dense(units=hp.Int(f'dense_units_{i}', min_value=128, max_value=512, step=128))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(rate=hp.Choice(f'dropout_post_dense_{i}_rate', values=[0.0,0.5]))(x)

    # ---------------------------

    output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

    # Connect input and output through the Model class
    model = Model(inputs=input_layer, outputs=output_layer)

    # best lr
    # hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

    # Return the model
    return model

In [ ]:
tuner = kt.BayesianOptimization(
    hypermodel=model_builder,
    objective='val_accuracy',
    directory=LOG_DIR,
    max_trials=30, # 10 default
    project_name='BayOpt')

In [ ]:
tuner.search(X_train,
             y_train,
             epochs=30,
             validation_data=(X_val,y_val))

In [ ]:
tuner.search_space_summary()

Get the optimal hyperparameters.

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
best_hp.values

Get the model with the optimal hyperparameters

In [ ]:
model = tuner.hypermodel.build(best_hp)
model.summary()

Train it completely.

In [ ]:
history, model = train_model(model)

In [ ]:
plot_history(history, save=True, filename='history_tuner', show_lr=False)

In [ ]:
evaluate_model(model, X_val, y_val)

In [ ]:
model.save(MODELS_DIR / 'tuner-0.7452')

In [ ]:
model = keras.models.load_model(MODELS_DIR / 'tuner-0.7452')

### Inception Time

In [ ]:
def inception_module(input_layer, bottleneck_size, filters, kernel_size, activation,stride=1):
    
    input_inception = Conv1D(filters=bottleneck_size, kernel_size=1,
                             padding='same', activation=activation, use_bias=False)(input_layer)
    
    kernel_size_s = [kernel_size // (2 ** i) for i in range(3)]
    conv_list = []
    for i in range(len(kernel_size_s)):
        conv_list.append(Conv1D(filters=filters, kernel_size=kernel_size_s[i],
                                strides=stride, padding='same', activation=activation, use_bias=False)(input_inception))
    
    max_pool = MaxPooling1D(pool_size=3, strides=stride, padding='same')(input_layer)
    
    conv = Conv1D(filters=filters, kernel_size=1,
                    padding='same', activation=activation, use_bias=False)(max_pool)
    
    conv_list.append(conv)

    x = Concatenate(axis=2)(conv_list)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    return x


def shortcut_layer(input_layer, output_layer):
    y = Conv1D(filters=int(output_layer.shape[-1]), kernel_size=1,
               padding='same', use_bias=False)(input_layer)
    y = BatchNormalization()(y)
    x = Add()([y,output_layer])
    x = Activation('relu')(x)
    return x

In [ ]:
def build_InceptionTime_classifier(depth, bottleneck_size, filters, kernel_size, activation, aug=False):

    input_layer = Input(shape=INPUT_SHAPE)
    x = input_layer
    
    if aug:
        x = data_augmentation()(x)
    
    input_res = x

    # inception block
    for d in range(depth):
        x = inception_module(x, bottleneck_size, filters, kernel_size, activation)
        if d%3 == 2:
            x = shortcut_layer(input_res,x)
            input_res = x

    x = GlobalAveragePooling1D()(x)
    x = Dropout(.5)(x)

    # Classifier
    x = Dense(256, activation='relu')(x)
    x = Dropout(.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(.5)(x)
    
    output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

    # Connect input and output through the Model class
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy', run_eagerly=aug)

    # Return the model
    return model

In [ ]:
model = build_InceptionTime_classifier(depth=6, bottleneck_size=32, filters=128, kernel_size=40, activation='tanh', aug=AUGMENTATION)
model._name = 'inception_time'
#model.summary()

In [ ]:
keras.utils.plot_model(model, to_file=REPORT_DIR / 'model_inception_time.pdf', show_shapes=True, show_layer_names=True)

In [ ]:
history, model = train_model(model)

In [ ]:
plot_history(history, save=True, filename='history_inception', show_lr=False)

In [ ]:
evaluate_model(model, X_val, y_val)

In [ ]:
model.save(MODELS_DIR / 'inception-0.7616.h5')

In [ ]:
model = keras.models.load_model(MODELS_DIR / 'inception-0.7616.h5', custom_objects={'data_augmentation': data_augmentation})

### Inception Time Ensemble

In [ ]:
CRAZY_ENSEMBLE_DIR = MODELS_DIR / 'inception_weighted'
CRAZY_ENSEMBLE_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# prepare the 12 models

for classToImprove in range(12):
    
    # build model
    model = build_InceptionTime_classifier(depth=6, bottleneck_size=32, filters=128, kernel_size=40, activation='tanh', aug=AUGMENTATION)

    # assign weight
    class_weights = dict(zip(range(0,12),np.ones(12)))
    class_weights[classToImprove] = 10

    # train model
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val,y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=[
            keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=10, factor=0.5, min_lr=1e-5)
        ]
    )

    # save model
    filename = f'inception_weighted_class_{classToImprove}.h5'
    model.save(CRAZY_ENSEMBLE_DIR / filename)

In [ ]:
label_mapping_flip = dict((v,k) for k,v in label_mapping.items())
label_mapping_flip

In [ ]:
# show accuracy of each model
for i in range(12):
    filename = f'inception_weighted_class_{i}.h5'
    model = keras.models.load_model(CRAZY_ENSEMBLE_DIR / filename, custom_objects={'data_augmentation': data_augmentation})
    predictions = model.predict(X_val, verbose=0)
    acc = accuracy_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1))
    print(f'Model tuned on class {label_mapping_flip[i]:<14} | Accuracy: {acc}')

In [ ]:
# load models
model_inception_base = keras.models.load_model(MODELS_DIR / 'inception-0.7616.h5', custom_objects={'data_augmentation': data_augmentation})
models = [model_inception_base]
for i in range(12):
    filename = f'inception_weighted_class_{i}.h5'
    model = keras.models.load_model(CRAZY_ENSEMBLE_DIR / filename, custom_objects={'data_augmentation': data_augmentation})
    model._name = f'model_class_{i}'
    models.append(model)

In [ ]:
# build ensemble
inputs = Input(shape=INPUT_SHAPE)
outputs = [model(inputs) for model in models]
outputs_ensemble = Average()(outputs)
model_ensemble = Model(inputs=inputs, outputs=outputs_ensemble, name='ensemble')

In [ ]:
# keras.utils.plot_model(model_ensemble, to_file=REPORT_DIR / 'model_inception_ensemble.pdf', show_shapes=True, show_layer_names=True)

In [ ]:
evaluate_model(model_ensemble, X_val, y_val, save=True, filename='confusion_crazy_ensemble')

In [ ]:
model_ensemble.save(MODELS_DIR / 'inception_crazy_ensemble-0.7699.h5')

In [ ]:
model_ensemble = keras.models.load_model(MODELS_DIR / 'inception_crazy_ensemble-0.7699.h5', custom_objects={'data_augmentation': data_augmentation})